<a href="https://colab.research.google.com/github/johnmelwin/ResearchProject1/blob/main/3_GPT_API_Call.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
!pip install config

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

## **Testing API**

In [ ]:
from openai import OpenAI

client = OpenAI()
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Give me 3 ideas for apps I could build with openai apis",
        }
    ],
    model="ft:gpt-3.5-turbo-1106:personal:jhot:9GizTjiH",
)
print(chat_completion.choices[0].message.content)

1. Virtual Assistant App: Build a virtual assistant app that utilizes OpenAI's natural language processing capabilities to help users with everyday tasks such as scheduling appointments, finding information, setting reminders, and more. Users could interact with the virtual assistant through voice commands or text input, making it easy to access information and complete tasks hands-free.

2. Writing Assistant App: Create a writing assistant app that leverages OpenAI's text generation capabilities to help users improve their writing skills. The app could provide suggestions for improving grammar, syntax, and style, as well as generate ideas for content and help users overcome writer's block. Users could also use the app to generate automatic summaries of long articles or documents, making it a valuable tool for students, writers, and professionals alike.

3. Mental Health Support App: Develop a mental health support app that utilizes OpenAI's sentiment analysis capabilities to provide p

# **Making API call for binary classification and exporting the output from GPT-3.5-turbo**

In [ ]:
from openai import OpenAI
import json
import csv
import numpy as np

client = OpenAI()

# Path to your JSONL file
jsonl_file_path = "/content/GPT4_testing_dataset_binary_JHOT_1000.jsonl"

# CSV file path
csv_file_path = "/content/GPT3.5PRETRAINED_JHOT_TEST.csv"

# Function to make a chat completion API call to GPT-3.5-turbo
def query_gpt_chat(messages):
    try:
        chat_completion = client.chat.completions.create(
            messages=messages,
            model="gpt-3.5-turbo",
            logprobs=True,
            top_logprobs=2,  # Fetch log probabilities for the top 10 tokens
        )
        # Retrieve the log probabilities if needed:
        log_probs = chat_completion.choices[0].logprobs.content[0].top_logprobs
        return log_probs[0].token, log_probs[0].logprob
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None

# Initialize message count and set maximum messages
message_count = 0
max_messages = 999

# Open the CSV file for writing
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    # Write the header
    csv_writer.writerow(["ID", "Input to the API", "Output from the API", "Log Probabilities", "Actual"])

    with open(jsonl_file_path, 'r', encoding='utf-8') as file:
        for line in file:
            if message_count >= max_messages:
                break  # Stop if the limit is reached
            entry = json.loads(line)

            user_messages = entry["messages"]
            actual_content = [message["content"] for message in entry["messages"] if message["role"] == "assistant"][0]

            # Convert the user_messages to a string format for the CSV
            input_to_api = json.dumps(user_messages)

            # Make a chat completion API call to GPT-3.5-turbo
            response_content, log_probs = query_gpt_chat(user_messages)
            prob = (np.exp(log_probs) * 100)

            # Increment message count
            message_count += 1
            print(f"Processed Message Count: {message_count}")
            print(response_content)
            print(prob)

            # Write the ID, input, output, log probabilities, and actual content to the CSV file
            csv_writer.writerow([message_count, input_to_api, response_content, json.dumps(prob), actual_content])


Processed Message Count: 1
False
98.65466749045991
Processed Message Count: 2
False
99.73184417178409
Processed Message Count: 3
False
97.97638899185804
Processed Message Count: 4
False
99.83306184700538
Processed Message Count: 5
False
97.91171839649034
Processed Message Count: 6
False
97.22483315682027
Processed Message Count: 7
False
91.71538458214162
Processed Message Count: 8
False
99.47411040652327
Processed Message Count: 9
True
79.36907954234417
Processed Message Count: 10
False
99.50946427552724
Processed Message Count: 11
False
76.61049562688493
Processed Message Count: 12
False
98.56673239072207
Processed Message Count: 13
False
96.06450529576854
Processed Message Count: 14
False
50.737966981122284
Processed Message Count: 15
False
98.34067402509255
Processed Message Count: 16
False
98.99134073522006
Processed Message Count: 17
False
96.1366508793678
Processed Message Count: 18
False
99.88449702609337
Processed Message Count: 19
True
90.82543622269856
Processed Message Count

# **Use sentiment analysis to identify the output from API text as TRUE or FLASE**


In [ ]:
import pandas as pd
from transformers import pipeline

# Initialize the sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

def classify_text(text):
    """
    Classifies text as True or False based on sentiment analysis.
    - Positive sentiment is considered True.
    - Negative sentiment is considered False.
    """
    result = sentiment_pipeline(text)[0]
    if result['label'] == 'POSITIVE' and result['score'] >= 0.5:
        return True
    elif result['label'] == 'NEGATIVE' and result['score'] >= 0.5:
        return False
    else:
        return None

# Load the CSV file
file_path = '/content/output_responses.csv'  # Update this path to your CSV file location
data = pd.read_csv(file_path)

# Apply the classification function to the "Output from the API" column
data['Predicted'] = data['Output from the API'].apply(classify_text)

# Save the updated DataFrame to a new CSV file
new_file_path = 'Predicted_Output.csv'  # Define your output file name
data.to_csv(new_file_path, index=False)

print(f"Updated DataFrame saved to {new_file_path}")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Updated DataFrame saved to Predicted_Output.csv


In [ ]:
import pandas as pd

# Load the new dataset
df = pd.read_csv('/content/GPT3.5_JHOT_TEST.csv')

# Display the first few rows of the dataframe to understand its structure
df.head()

from sklearn.metrics import confusion_matrix, classification_report, fbeta_score
import seaborn as sns
import matplotlib.pyplot as plt

# Since the 'BERT' column represents the prediction, let's rename it to 'BERT_Prediction' to match the code's logic.
df = df.rename(columns={'BERT': 'BERT_Prediction'})

# Convert columns to boolean
df['Actual'] = df['Actual'].astype(bool)
df['BERT_Prediction'] = df['BERT_Prediction'].astype(bool)
df['TUNED_GPT'] = df['TUNED_GPT'].astype(bool)

# Generate and print classification reports
models = {
    'BERT': 'BERT_Prediction',
    'TUNED_GPT': 'TUNED_GPT'
}

for model_name, model_column in models.items():
    report = classification_report(df['Actual'], df[model_column], target_names=['False', 'True'])
    print(f"Classification Report for {model_name}:")
    print(report)
    print("-----------------------------------------------")

    # Calculate F2 score
    f2_score = fbeta_score(df['Actual'], df[model_column], beta=2.0, zero_division=1)
    print(f"F2 Score for {model_name}:", f2_score)
    print("-----------------------------------------------")

    # Generate and print confusion matrix
    cm = confusion_matrix(df['Actual'], df[model_column])
    print(f"Confusion Matrix for {model_name}:")
    print(cm)
    print("-----------------------------------------------")

    # Visualize the confusion matrix
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['False', 'True'], yticklabels=['False', 'True'])
    plt.title(f'Confusion Matrix for {model_name} Predictions')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()
